Solutions Sheet 1.2: RSA with politeness
========================================

**Author:** Michael Franke



In [1]:
##################################################
## imports
##################################################

import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

##################################################
## helper functions
##################################################

def softmax(x, axis=1):
    """
    Softmax function in numpy
    Parameters
    ----------
    x: array
        An array with any dimensionality
    axis: int
        The axis along which to apply the softmax
    Returns
    -------
    array
        Same shape as x
    """
    e_x = np.exp(x - np.max(x, axis, keepdims=True))
    return e_x / e_x.sum(axis=axis, keepdims=True)


def normalize(arr, axis=1):
    """
    Normalize arr along axis
    """
    return arr / arr.sum(axis, keepdims=True)

##################################################
## defining the context
##################################################

states     = [1,2,3,4,5]
utterances = ["terrible","bad","okay","good","amazing"]

semantic_meaning = np.array(
    [[.95 ,.85 ,.02 ,.02,.02],    # terrible
     [.85 ,.95 ,.02 ,.02,.02],    # bad
     [.02 ,.25 ,.95 ,.65,.35],    # okay
     [.02 ,.05 ,.55 ,.95,.93],    # good
     [.02 ,.02 ,.02 ,.65,.95]]    # amazing
)

> <strong><span style=&ldquo;color:#D83D2B;&rdquo;>Exercise 1.2.1: Check experiment and your intuitions</span></strong>
>
> 1. Consult the original paper ([Yoon et al. 2016](http://langcog.stanford.edu/papers_new/yoon-2016-cogsci.pdf)) to find the description of the experiment that was used to get these semantic values. Describe this experiment in at most three simple sentences: what was the question participants had to answer and how were answers recorded?
> 2. Comment on whether you find the obtained values intuitive as values of the semantic meaning of these expression.
> 3. Do you think that the experiment was well-designed for the task of eliciting information about semantic meaning of expressions?

> <strong><span style=&ldquo;color:#D83D2B;&rdquo;>Solutions for Exercise 1.2.1: Check experiment and your intuitions</span></strong>
>
> 1. The experiment elicited binary judgements (&rsquo;yes&rsquo; and &rsquo;no&rsquo;) from participants. Each task presented two things: (i) a true state (Bob&rsquo;s true feeling about some item, shown as 1 to 5 out of 5 hearts), and (ii) a question &ldquo;Do you think that Bob thought Ann&rsquo;s cake was X?&rdquo; where X was one of the target words. The proportion of &rsquo;yes&rsquo; answers for each pair of (i) number of hearts and (ii) target word were used as the literal semantic value of that pair.
> 2. The resulting values seem intuitive enough, at least when we look at the number of stars that received the highest rating for each word.
> 3. The procedure is okay, as it is difficult to elicit semantic (truth) values. But it seems that this procedure (like many others) may slightly conflate the genuine underlying semantic meaning with a pragmatically enriched interpretation.



In [1]:
##################################################
## model parameters
##################################################

alpha        = 10
phi          = 0.99
social_value = 1.25

##################################################
## RSA speaker with politeness
##################################################

def RSA_polite_speaker(alpha, phi, social_value):
    """
    predictions of an RSA model with politeness (speaker part)
    (following: http://www.problang.org/chapters/09-politeness.html)
    Parameters
    ----------
    alpha: float
        Optimality parameter
    phi: float
        Relative weight of epistemic utility component
    social_value: float
        Social value factor (how much more "socially valuable" is one more star?)
    Returns
    -------
    array
        probability that speaker chooses utterance for each state
    """
    literal_listener   = normalize(semantic_meaning)
    epistemic_utility  = np.log(np.transpose(literal_listener))
    social_utility     = np.sum(literal_listener * np.array([states]) * social_value, axis=1)
    util_speaker       = phi * epistemic_utility + (1-phi) * social_utility
    pragmatic_speaker  = softmax(alpha * util_speaker)
    return(pragmatic_speaker)

RSA_speaker_predictions = RSA_polite_speaker(alpha, phi, social_value)

##################################################
## showing and plotting the results
##################################################

speaker  = pd.DataFrame(data    = RSA_speaker_predictions,
                        index   = states,
                        columns = utterances)
speaker['object'] = speaker.index

print(speaker.round(4))

speaker_long = speaker.melt(id_vars      = "object",
                            var_name     = "utterance",
                            value_name   = "probability",
                            ignore_index = False)
speaker_plot = sns.FacetGrid(speaker_long, col="object")
speaker_plot.map(sns.barplot, "utterance", "probability")
# plt.show()

terrible     bad    okay    good  amazing  object
1    0.6804  0.3196  0.0000  0.0000   0.0000       1
2    0.2641  0.7355  0.0004  0.0000   0.0000       2
3    0.0000  0.0000  0.9780  0.0220   0.0000       3
4    0.0000  0.0000  0.0059  0.2180   0.7761       4
5    0.0000  0.0000  0.0000  0.0113   0.9887       5

> <strong><span style=&ldquo;color:#D83D2B;&rdquo;>Exercise 1.2.2: Explore the polite speaker model</span></strong>
>
> 0. For yourself, in order to understand the model, go through each line of the definition of the function \`RSA<sub>polite</sub><sub>speaker</sub>\` and make sure that you understand what is happening.
> 1. Change the call to the speaker to make it so that it only cares about making the listener feel good. What parameter value(s) did you choose?
> 2. Change the call to the speaker to make it so that it cares about both making the listener feel good *and* conveying information. What parameter value(s) did you choose?
> 3. If we set $\varphi=1$, and choose a very high $\alpha$, the speaker behavior is quite regular. Is this kind of behavior intuitive? Do you think it happens in real life?
> 4. Is there a parameter setting for this model, such that the speaker would at most choose one category higher when trying to be polite? For example, when they should informatively say &rsquo;okay&rsquo;, they would say &rsquo;good&rsquo;, but never &rsquo;amazing&rsquo;. Which parameters achieve this? Or, how could the model be changed to achieve this behavior?

> <strong><span style=&ldquo;color:#D83D2B;&rdquo;>Solutions Exercise 1.2.2: Explore the polite speaker model</span></strong>
>
> 1. To make the speaker only consider social utility, we set &ldquo;phi = 0&rdquo;.
> 2. Any value of parameter &ldquo;phi&rdquo; that is not 0 or 1 takes both types of utility into account.
> 3. Setting $\varphi=1$, and choose a very high $\alpha = 10,000$, the speaker only cares about informativity and is extremely &ldquo;rational&rdquo; (in the technical sense of utility maximization). It&rsquo;s dubious that such agents are implementable in neural wetware, and the result is rather extreme: there is exactly one word the speaker would send with near certainty in each state. Interestingly, the speaker is predicted to use &ldquo;amazing&rdquo; for both 4 and 5 stars, and not to use &ldquo;good&rdquo; at all. That is clearly unintuitive from a global perspective, because it is not optimal for achieving perfect communication. But it is what follows from the assume semantics. Since &ldquo;good&rdquo; got a rating of .55 for the 3 star-state, after normalization (literal listener), the probability to single out the 4 star state is higher with &ldquo;amazing&rdquo; than it is with &ldquo;good&rdquo;.



In [1]:
alpha = 10000
phi   = 1
RSA_speaker_predictions = RSA_polite_speaker(alpha, phi, social_value)

##################################################
## showing and plotting the results
##################################################

speaker  = pd.DataFrame(data    = RSA_speaker_predictions,
                        index   = states,
                        columns = utterances)
speaker['object'] = speaker.index

print(speaker.round(2))

speaker_long = speaker.melt(id_vars      = "object",
                            var_name     = "utterance",
                            value_name   = "probability",
                            ignore_index = False)
speaker_plot = sns.FacetGrid(speaker_long, col="object")
speaker_plot.map(sns.barplot, "utterance", "probability")
plt.show()

terrible  bad  okay  good  amazing  object
1       1.0  0.0   0.0   0.0      0.0       1
2       0.0  1.0   0.0   0.0      0.0       2
3       0.0  0.0   1.0   0.0      0.0       3
4       0.0  0.0   0.0   0.0      1.0       4
5       0.0  0.0   0.0   0.0      1.0       5

> 4. This is difficult with this model, if possible at all. We need a rather low value of $\varphi$ to get the speaker to use &ldquo;good&rdquo; for sate 2, but then as $\varphi$ gets lower and lower, the speaker starts to use &ldquo;amazing&rdquo; for state 1. The problem is that social utility is about making the speaker feel as good as possible, not just as &ldquo;a bit better than the true state&rdquo;. We could define an alternative social utility function which achieves this better than the current model. It would be an interesting empirical question which kind of social utility function best explains human judgements about polite language use. (Your project?)



In [1]:
##################################################
## pragmatic listener infers politeness level
##################################################

# which phi-values to consider
phi_marks     = np.linspace(start=0, stop=1, num=11)
phi_prior_flt = np.array([1,1,1,1,1,1,1,1,1,1,1])   # flat
phi_prior_bsd = np.array([1,2,3,4,5,6,7,8,9,10,11]) # biased towards politeness

def RSA_polite_listener(alpha, phi_prior, social_value):
    """
    predictions of an RSA model with politeness (listener part)
    (following: http://www.problang.org/chapters/09-politeness.html)
    Parameters
    ----------
    alpha: float
        Optimality parameter
    phi_priors: float
        Prior over degree of politeness (phi-parameter)
    social_value: float
        Social value factor (how much more "socially valuable" is one more star?)
    Returns
    -------
    array
         for each message: listener posterior over state-phi pairs
    """
    phi_prior = phi_prior / np.sum(phi_prior) # make sure priors are normalized
    posterior = np.zeros((len(utterances), len(states),len(phi_marks)))
    for i in range(len(phi_marks)):
        pragmatic_speaker   = RSA_polite_speaker(alpha, phi_marks[i], social_value)
        posterior_given_phi = normalize(np.transpose(pragmatic_speaker), axis=1)
        posterior[:,:,i]    = posterior_given_phi * phi_prior[i]
    return(posterior)

RSA_listener_predictions = RSA_polite_listener(alpha, phi_prior_bsd, social_value)

print("listener posterior over states after hearing 'good':\n",
      np.sum(RSA_listener_predictions[3,:,:], axis=1))

iterables=[utterances, states, phi_marks]
index = pd.MultiIndex.from_product(iterables, names=['utterances','states','phi'])

listener = pd.DataFrame(RSA_listener_predictions.reshape(RSA_listener_predictions.size, 1),
                        index=index)
listener = listener.reset_index()

##################################################
## plotting the results
##################################################

def plot_listener(utterance_index):
    print("plotting listener posterior for utterance:", utterances[utterance_index])
    predictions = RSA_listener_predictions[utterance_index,:,:]
    sns.heatmap(predictions)
    plt.show()

plot_listener(3)

listener posterior over states after hearing 'good':
 [0.00588069 0.01970965 0.40323864 0.53989896 0.03127205]
plotting listener posterior for utterance: good

> <strong><span style=&ldquo;color:#D83D2B;&rdquo;>Exercise 1.2.3: Explore the pragmatic listener</span></strong>
>
> 0. For yourself, in order to understand the model, go through each line of the definition of the function \`RSA<sub>polite</sub><sub>listener</sub>\` and make sure that you understand what is happening.
> 1. What does the heatmap show? What&rsquo;s on the x-axis, what&rsquo;s on the y-axis, and what do the colors mean?
> 2. Add a function that takes an utterance index (0, &#x2026;, 4) and outputs three things: (i) a print out of the [marginal distribution](https://en.wikipedia.org/wiki/Marginal_distribution) over states, (ii) a print out of the marginal distribution over $\varphi$ values, and (iii) the heatmap visualizing the joint-distribution of both.
> 3. Compare the interpretation of the utterance &rsquo;amazing&rsquo; with that of the other utterances (for the parameter values used originally). Explain in what sense the distribution shown for &rsquo;amazing&rsquo; is a [multimodal distribution](https://en.wikipedia.org/wiki/Multimodal_distribution). Explain why the model makes this multi-modal prediction for &rsquo;amazing&rsquo;. Does it also predict multi-modality for &rsquo;good&rsquo;? What about &rsquo;terrible&rsquo;?

> <strong><span style=&ldquo;color:#D83D2B;&rdquo;> Solutions Exercise 1.2.3: Explore the pragmatic listener</span></strong>
>
> 1. The heatmap has the five states (star ratings) on the $y$-axis, and the 11 values for $\varphi$ that we consider here on the $x$ axis. The color coding shows the joint posterior probability for each pair of state and $\varphi$ value.
> 2. Here is a function that: (i) a print out of the [marginal distribution](https://en.wikipedia.org/wiki/Marginal_distribution) over states, (ii) a print out of the marginal distribution over $\varphi$ values, and (iii) the heatmap visualizing the joint-distribution of both. This function directly computes the predictions, based on parameter input. This is for convenience (not strictly required from the exercise).



In [1]:
def myFun(utterance_index, alpha, phi_prior, social_value):
    RSA_listener_predictions = RSA_polite_listener(alpha, phi_prior, social_value)

    iterables=[utterances, states, phi_marks]
    index = pd.MultiIndex.from_product(iterables, names=['utterances','states','phi'])

    listener = pd.DataFrame(RSA_listener_predictions.reshape(RSA_listener_predictions.size, 1),
                            index=index)
    listener = listener.reset_index()

    # marginal over states
    print("listener posterior over states after hearing '%s':\n" % utterances[utterance_index],
          np.sum(RSA_listener_predictions[utterance_index,:,:], axis=1).round(3))
    # marginal over phi values
    print("listener posterior over phi-values after hearing '%s':\n" % utterances[utterance_index],
          np.sum(RSA_listener_predictions[utterance_index,:,:], axis=0).round(3))
    # plot of joint-posterior
    predictions = RSA_listener_predictions[utterance_index,:,:]
    sns.heatmap(predictions)
    plt.show()

myFun(1, alpha, phi_prior_bsd, social_value)

listener posterior over states after hearing 'bad':
 [0.43  0.56  0.003 0.003 0.003]
listener posterior over phi-values after hearing 'bad':
 [0.015 0.03  0.045 0.061 0.076 0.091 0.106 0.121 0.136 0.152 0.167]

> 3. Here is posterior for &ldquo;amazing&rdquo;:



In [1]:
myFun(4,alpha, phi_prior_bsd, social_value)

listener posterior over states after hearing 'amazing':
 [0.072 0.038 0.01  0.386 0.494]
listener posterior over phi-values after hearing 'amazing':
 [0.015 0.03  0.045 0.061 0.076 0.091 0.106 0.121 0.136 0.152 0.167]

And here it is for &ldquo;good&rdquo;:



In [1]:
myFun(3,alpha, phi_prior_bsd, social_value)

listener posterior over states after hearing 'good':
 [0.006 0.02  0.403 0.54  0.031]
listener posterior over phi-values after hearing 'good':
 [0.015 0.03  0.045 0.061 0.076 0.091 0.106 0.121 0.136 0.152 0.167]

And &ldquo;terrible&rdquo;



In [1]:
myFun(0,alpha, phi_prior_bsd, social_value)

listener posterior over states after hearing 'terrible':
 [0.746 0.245 0.003 0.003 0.003]
listener posterior over phi-values after hearing 'terrible':
 [0.015 0.03  0.045 0.061 0.076 0.091 0.106 0.121 0.136 0.152 0.167]

While there are also two local peaks in the posterior of &ldquo;good&rdquo;, the case of &ldquo;amazing&rdquo; is most clearly multimodal.
One interpretation possibility is that the speaker really found the cookies terrible (1 star), but wanted didn&rsquo;t want to throw informativity overboard entirely ($\varphi$ index 4, state index 0).
Another peak possibility is that the cookies really were amazing and that the speaker is maximally informative.
(NB: the latter is a peak because of the biased prior.)

